**これは完成したプログラムを実際に適用するファイルなので、大幅な変更を行う場合はコピーを作ろう**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## For Sign Recognition Training

### import from localdata

In [2]:
from google.colab import files
files.upload()

Saving train_dir3class-balanced.zip to train_dir3class-balanced.zip


KeyboardInterrupt: ignored

In [0]:
!unzip train_dir3class-balanced.zip

In [0]:
ls

 drive/            'train_dir3class-balanced (1).zip'
 sample_data/       train_dir3class-balanced.zip
 train_dir3class/


In [0]:
# %cp -r "/content/drive/My Drive/3is_project/GTRSB_kaggle/train_dir" "./"

###転移学習

In [0]:
import torch
print(torch.__version__)
import torchvision
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import numpy as np

# print(PIL.__version__)

1.3.1


### imagefolder dataloader cv2_imshow

In [0]:

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((64,64)),
        transforms.RandomAffine(5,shear=20,fillcolor=(127,127,127)),
        transforms.RandomApply([transforms.CenterCrop(51)],0.7),
        transforms.RandomApply([transforms.Pad(10)],0.7),
        transforms.Resize((64,64)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = './train_dir3class'
train_dataset = datasets.ImageFolder(data_dir,data_transforms['train'])
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=256,shuffle=True,num_workers=4)

In [0]:
from  google.colab.patches import cv2_imshow
import cv2
for i ,(images,labels) in enumerate(train_loader):
  # print(labels.numpy())
  # img = torchvision.utils.make_grid(images,padding=1)
  img = images[0]
  img = img.numpy()
  img = np.transpose(img,(1,2,0))
  # print(images.shape)
  # print(images[0])
  cv2_imshow(cv2.cvtColor(img*255,cv2.COLOR_RGB2BGR))
  if(i>40):
    break

### train

In [0]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [0]:
class My_model(nn.Module):
  def __init__(self):
    super(My_model,self).__init__()
    self.vgg = models.vgg19_bn(pretrained=True)
    for param in self.vgg.features[0:26].parameters():
       param.requires_grad = False
    for param in self.vgg.features[26:53].parameters():
       param.requires_grad = True
    num_ftrs = self.vgg.classifier[6].in_features
    self.vgg.classifier[6] = nn.Linear(num_ftrs, 3)
    
    
  def forward(self,x):
    x = self.vgg(x)
    # x = torch.sigmoid(x)
    return x


In [0]:
# model = models.vgg19(pretrained=True)
# model = models.vgg19(pretrained=False)
# model = model.to(device)
# model
model = My_model()
model = model.to(device)

In [0]:
for p in model.vgg.classifier.parameters():
  print(p.requires_grad)

In [0]:
model

In [0]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=5e-4)
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=5e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [0]:
import nvidia_smi
!nvidia-smi

In [0]:
!ps -elf | grep python

In [0]:
!ps aux
!kill -9 19

In [0]:
num_epochs = 5
train_loss_list = []
train_acc_list = []

for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    
    #train
    model.train()
    for i, (images, labels) in enumerate(train_loader):
      images,labels = images.to(device),labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      train_loss += loss.item()
      train_acc += (outputs.max(1)[1] == labels).sum().item()
      loss.backward()
      optimizer.step()
    
    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_acc = train_acc / len(train_loader.dataset)

    print ('Epoch [{}/{}], Loss: {loss:.4f}, Acc: {acc:.4f}, lr：{learning_rate}' 
                       .format(epoch+1, num_epochs , loss=avg_train_loss, acc=avg_train_acc,learning_rate=optimizer.param_groups[0]["lr"]))
    #学習率調整
    lr_scheduler.step()
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)

Epoch [1/5], Loss: 0.0025, Acc: 0.7769, lr：1e-05
Epoch [2/5], Loss: 0.0003, Acc: 0.9833, lr：1e-05
Epoch [3/5], Loss: 0.0001, Acc: 0.9956, lr：1e-05
Epoch [4/5], Loss: 0.0000, Acc: 0.9976, lr：1e-05
Epoch [5/5], Loss: 0.0000, Acc: 0.9997, lr：1e-05


### model save ,load

In [0]:
torch.save(model.state_dict(), '/content/drive/My Drive/3is_project/GTRSB_kaggle/models/12_17_fined.model')

In [0]:
# 12_11_fined3 -> sigmoid;  12_11_fined2 -> linear
model = My_model()
param = torch.load('/content/drive/My Drive/3is_project/GTRSB_kaggle/models/12_17_fined.model')
model.load_state_dict(param)

<All keys matched successfully>

In [0]:
ls '/content/drive/My Drive/3is_project/GTRSB_kaggle/models/'

12_11_fined2.model  12_15_fined.model      class3_12_6_fined.h5
12_11_fined3.model  class3_11_28_fined.h5  test2.model
12_11_fined.model   class3_11_28.h5        test.model


## area detection

### image cuts

In [0]:
def upCut(path,verbose=False):
  import numpy as np
  import cv2
  import time
  if type(path) == str:
    start= time.time()
    img = cv2.imread(path)
    print(f"file load:{time.time() - start}")
    img = cv2.resize(img , dsize=(640, 480))
  else: 
    img = path
  
  img = img[0:int(len(img)/3),:,:]
  if verbose:
   cv2.imshow(img)
  return img

def rightCut(path,ratio,idx,verbose=False):
  from google.colab.patches import cv2_imshow
  import numpy as np
  import cv2
  if type(path) == str:
    img = cv2.imread(path)
    img = cv2.resize(img , dsize=(640, 480))
  else: 
    img = path 
  img = img[:,int(len(img[0])/ratio)*idx:int(len(img[0])/ratio)*(idx+1),:]
  if verbose:
    cv2_imshow(img)
  return img

### simplize

In [0]:
def simplize(path,verbose=False): 
  from google.colab.patches import cv2_imshow
  import numpy as np
  import cv2
  if type(path) == str:
    img = cv2.imread(path)
    img = cv2.resize(img , dsize=(640, 480))
  else: 
    img = path
  ## HSVFULLじゃないことに注意！！
  hsv = cv2.cvtColor(img ,cv2.COLOR_BGR2HSV)
  ## red 160 - 19 yellow 20-39 green 40 - 79 cyan 80 - 99 blue 100 - 139  biolet 140-159  
  hsv[:,:,0]=np.where( (160 <= hsv[:,:,0]) | (hsv[:,:,0] <= 19),0,hsv[:,:,0])#red
  hsv[:,:,0]=np.where( (20 <= hsv[:,:,0]) & (hsv[:,:,0] <= 39),30,hsv[:,:,0])#yellow
  hsv[:,:,0]=np.where( (40 <= hsv[:,:,0]) & (hsv[:,:,0] <= 79),60,hsv[:,:,0])#green
  hsv[:,:,0]=np.where( (80 <= hsv[:,:,0]) & (hsv[:,:,0] <= 99),90,hsv[:,:,0])#cyan
  hsv[:,:,0]=np.where( (100 <= hsv[:,:,0]) & (hsv[:,:,0] <= 139),120,hsv[:,:,0])#blue
  hsv[:,:,0]=np.where( (140 <= hsv[:,:,0]) & (hsv[:,:,0] <= 159),150,hsv[:,:,0])#violet
  ### 彩度の範囲を広げると白色が検出しづらくなる。明度を広げると薄い色が検出しづらくなる。
  hsv[:,:,1]=np.where( (hsv[:,:,1] <= 60) & (160 <= hsv[:,:,2]  )  ,0,hsv[:,:,1]) #white 
  hsv[:,:,2]=np.where( hsv[:,:,2] <=  100, 0,hsv[:,:,2]) #black # flatten all colors
  if verbose:
    cv2_imshow(cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR))
  return cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)


In [0]:
def rb_filter(path,verbose=False):
  if type(path) == str:
    img = cv2.imread(path)
    img = cv2.resize(img , dsize=(640, 480))
  else: 
    img = path
  if verbose:
    cv2_imshow(img)
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  hsv[:,:,2]=np.where( ((hsv[:,:,0] == 0)|(hsv[:,:,0]==150)|(hsv[:,:,0] == 120)|(hsv[:,:,0]==90)) & (hsv[:,:,1]  > 30 ), 255,0)
  img = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
  if verbose:
    cv2_imshow(img)
  return img


###selective search

In [0]:
import skimage.io
import skimage.feature
import skimage.color
import skimage.transform
import skimage.util
import skimage.segmentation
import numpy
import time

def _generate_segments(im_orig, scale, sigma, min_size):
    """
        segment smallest regions by the algorithm of Felzenswalb and
        Huttenlocher
    """

    # open the Image
    im_mask = skimage.segmentation.felzenszwalb(
        im_orig, scale=scale, sigma=sigma,
        min_size=min_size)

    # merge mask channel to the image as a 4th channel

    im_orig = numpy.append(
        im_orig, numpy.zeros((len(im_orig),len(im_orig[0])))[:, :, numpy.newaxis], axis=2)
    im_orig[:, :, 3] = im_mask
    return im_orig


def _sim_colour(r1, r2):
    """
        calculate the sum of histogram intersection of colour
    """
    return sum([min(a, b) for a, b in zip(r1["hist_c"], r2["hist_c"])])


def _sim_texture(r1, r2):
    """
        calculate the sum of histogram intersection of texture
    """
    return sum([min(a, b) for a, b in zip(r1["hist_t"], r2["hist_t"])])


def _sim_size(r1, r2, imsize):
    """
        calculate the size similarity over the image
    """
    return 1.0 - (r1["size"] + r2["size"]) / imsize


def _sim_fill(r1, r2, imsize):
    """
        calculate the fill similarity over the image
    """
    bbsize = (
        (max(r1["max_x"], r2["max_x"]) - min(r1["min_x"], r2["min_x"]))
        * (max(r1["max_y"], r2["max_y"]) - min(r1["min_y"], r2["min_y"]))
    )
    return 1.0 - (bbsize - r1["size"] - r2["size"]) / imsize


def _calc_sim(r1, r2, imsize):
    return (_sim_colour(r1, r2) + _sim_texture(r1, r2)
            + _sim_size(r1, r2, imsize) + _sim_fill(r1, r2, imsize))


def _calc_colour_hist(img):
    BINS = 25
    hist = numpy.array([])
    for colour_channel in (0, 1, 2):
        # extracting one colour channel
        c = img[:, colour_channel]
        # calculate histogram for each colour and join to the result
        hist = numpy.concatenate(
            [hist] + [numpy.histogram(c, BINS, (0.0, 255.0))[0]])
    # L1 normalize
    hist = hist / len(img)

    return hist


def _calc_texture_gradient(img):
    ret = numpy.zeros((img.shape[0], img.shape[1], img.shape[2]))
    for colour_channel in (0, 1, 2):
        ret[:, :, colour_channel] = skimage.feature.local_binary_pattern(
            img[:, :, colour_channel], 8, 1.0)
    return ret


def _calc_texture_hist(img):
    BINS = 10
    hist = numpy.array([])

    for colour_channel in (0, 1, 2):

        # mask by the colour channel
        fd = img[:, colour_channel]

        # calculate histogram for each orientation and concatenate them all
        # and join to the result
        hist = numpy.concatenate(
            [hist] + [numpy.histogram(fd, BINS, (0.0, 1.0))[0]])

    # L1 Normalize
    hist = hist / len(img)

    return hist


def _extract_regions(img):
    R = {}
    hsv = skimage.color.rgb2hsv(img[:, :, :3])
    for y, i in enumerate(img):
        for x, (r, g, b, l) in enumerate(i):
            if l not in R:
                R[l] = {
                    "min_x": 0xffff, "min_y": 0xffff,
                    "max_x": 0, "max_y": 0, "labels": l}
            if R[l]["min_x"] > x:
                R[l]["min_x"] = x
            if R[l]["min_y"] > y:
                R[l]["min_y"] = y
            if R[l]["max_x"] < x:
                R[l]["max_x"] = x
            if R[l]["max_y"] < y:
                R[l]["max_y"] = y
    tex_grad = _calc_texture_gradient(img)
    for k, v in list(R.items()):
        masked_pixels = hsv[:, :, :][img[:, :, 3] == k]
        R[k]["size"] = len(masked_pixels / 4)
        R[k]["hist_c"] = _calc_colour_hist(masked_pixels)
        R[k]["hist_t"] = _calc_texture_hist(tex_grad[:, :][img[:, :, 3] == k])
    return R


def _extract_neighbours(regions):
    def intersect(a, b):
        if (a["min_x"] < b["min_x"] < a["max_x"]
                and a["min_y"] < b["min_y"] < a["max_y"]) or (
            a["min_x"] < b["max_x"] < a["max_x"]
                and a["min_y"] < b["max_y"] < a["max_y"]) or (
            a["min_x"] < b["min_x"] < a["max_x"]
                and a["min_y"] < b["max_y"] < a["max_y"]) or (
            a["min_x"] < b["max_x"] < a["max_x"]
                and a["min_y"] < b["min_y"] < a["max_y"]):
            return True
        return False
    R = list(regions.items())
    neighbours = []
    for cur, a in enumerate(R[:-1]):
        for b in R[cur + 1:]:
            if intersect(a[1], b[1]):
                neighbours.append((a, b))
    return neighbours
def _merge_regions(r1, r2):
    new_size = r1["size"] + r2["size"]
    rt = {
        "min_x": min(r1["min_x"], r2["min_x"]),
        "min_y": min(r1["min_y"], r2["min_y"]),
        "max_x": max(r1["max_x"], r2["max_x"]),
        "max_y": max(r1["max_y"], r2["max_y"]),
        "size": new_size,
        "hist_c": (
            r1["hist_c"] * r1["size"] + r2["hist_c"] * r2["size"]) / new_size,
        "hist_t": (
            r1["hist_t"] * r1["size"] + r2["hist_t"] * r2["size"]) / new_size,
        "labels": r1["labels"] + r2["labels"]
    }
    return rt
def selective_search(im_orig, scale=1.0, sigma=0.8, min_size=50):
    assert im_orig.shape[2] == 3, "3ch image is expected"
    img = _generate_segments(im_orig, scale, sigma, min_size)
    if img is None:
        return None, {}
    imsize = img.shape[0] * img.shape[1]
    R = _extract_regions(img)
    neighbours = _extract_neighbours(R)
    S = {}
    for (ai, ar), (bi, br) in neighbours:
        S[(ai, bi)] = _calc_sim(ar, br, imsize)
    while S != {}:
        i, j = sorted(S.items(), key=lambda i: i[1])[-1][0]
        t = max(R.keys()) + 1.0
        R[t] = _merge_regions(R[i], R[j])
        key_to_delete = []
        for k, v in list(S.items()):
            if (i in k) or (j in k):
                key_to_delete.append(k)
        for k in key_to_delete:
            del S[k]
        for k in [a for a in key_to_delete if a != (i, j)]:
            n = k[1] if k[0] in (i, j) else k[0]
            S[(t, n)] = _calc_sim(R[t], R[n], imsize)
    regions = []
    for r in R.values():
      w = r['max_x'] - r['min_x']
      h = r['max_y'] - r['min_y']
      ratio = h/w if w>=h else w/h
      if( ratio>=0.8 and w*h > 1000 and w*h < 5000):
        regions.append({
              'x_':r['min_x'], 
              'y_':r['min_y'],
              '_x':r['max_x'],
              '_y':r['max_y'],
              'area': w*h,
        })
    sort = sorted(regions, key=lambda x:x["area"],reverse=True)
    idx=0
    ## non maximum suppression
    for i1, r1 in enumerate(sort):
      idx = i1 + 1
      for r2 in sort[i1+1:]:
        if r1["x_"] <= r2["x_"] and r1["_x"] >= r2["_x"] and r1["y_"] <= r2["y_"] and r1["_y"] >= r2["_y"]:    
          del sort[idx]
          idx -= 1
        idx += 1 
    return img, sort

### road sign recog

In [0]:
device= "cuda"
model = model.eval()
model = model.to(device)
def classify_torch(model,detected,exam=True):
  from google.colab.patches import cv2_imshow
  import cv2
  import time
  if exam:
    cv2_imshow(detected)
  image = cv2.cvtColor(detected , cv2.COLOR_BGR2RGB)
  trf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64,64)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
  image=trf(image)
  image = image.view(1,3,64,64)
  image = image.float()
  image = image.to(device)
  
  start = time.time()
  output= model(image)
  print(f"recog time: {time.time()-start}")
  clas = pprint(output,exam)
  return clas

In [0]:
def classify(model,detected,exam=True):
  from google.colab.patches import cv2_imshow
  import cv2
  import time
  image = cv2.resize(detected , dsize=(64, 64))
  if exam:
    cv2_imshow(image)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
  image = image.reshape((1,) + image.shape)
  image = image / 255
  start = time.time()
  clas = pprint(model.predict(image),exam)
  print(f" recog time:{time.time() - start}")
  return clas


In [0]:
def pprint(arr,exam=True):
  print(f" {arr}")
  diff = abs(arr[0][0]-arr[0][1]-arr[0][2])
  idx = arr[0].argmax()
  if exam:
    print(f" difference = {diff}")
  if arr[0][idx]<5 or arr[0][(idx+1)%3]>0 or arr[0][(idx+2)%3]>0 :
  # if diff<0.7 :
    print(" Others")
    return 3
  elif arr.argmax() == 0:
    print(" Speed restriction 30")
    return 0
  elif arr.argmax() == 1:
    print(" GO")
    return 1
  elif arr.argmax() == 2:
    print(" STOP")
    return 2

**対象を一つ検出したら検索を切り上げる**

In [0]:
def s_search(path,exam=True):
  from google.colab.patches import cv2_imshow
  import numpy as np
  import cv2
  import time 
  print(f"\nimage path = {path}")
  s = time.perf_counter()
  cut = rightCut(upCut(pth),3,idx=0,verbose=False)
  img = simplize(cut,False)
  img = rb_filter(img,False)
  img = cv2.medianBlur(img,5)
  img = cv2.GaussianBlur(img,(17,17),0,0)
  print(f"maeshori:{time.perf_counter()-s}")
  print("_______SEARCH START_______")
  start = time.perf_counter()
  _, regions = selective_search(img.astype(np.float64), scale=3000, sigma=0.0, min_size=100)
  print(f" search time:{time.perf_counter() - start}")
  for r in regions:
    start = time.perf_counter()
    c = classify_torch(model,cut[r["y_"]:r["_y"],r["x_"]:r["_x"],:],exam)
    print(f"recog time:{time.perf_counter() - start}")
    if c!=3:
      return c
  return c
# pth = "/content/drive/My Drive/3is_project/GTRSB_kaggle/vali_dir00_speed/speed_img_000.jpg"
pth = "/content/drive/My Drive/3is_project/GTRSB_kaggle/micon_images/micon_img_320.jpg"
# pth = "/content/drive/My Drive/3is_project/GTRSB_kaggle/micon_images/micon_img_520.jpg"
# pth = "/content/drive/My Drive/3is_project/GTRSB_kaggle/output2.jpg"
# pth = "/content/drive/My Drive/3is_project/GTRSB_kaggle/win_images/20191201/win_img_000.jpg"
# pth = "/content/drive/My Drive/3is_project/GTRSB_kaggle/win_images/20191201/win_img_010.jpg"
# pth = "/content/drive/My Drive/3is_project/GTRSB_kaggle/win_images/20191201/win_img_020.jpg"
# pth = '/content/drive/My Drive/3is_project/GTRSB_kaggle/win_images/20191129_17_03_16/win_img_1600.jpg'
import time
start=time.perf_counter()
sign = s_search(pth,False)
print(f"about time:{time.perf_counter() -start}")
print(sign)


image path = /content/drive/My Drive/3is_project/GTRSB_kaggle/micon_images/micon_img_320.jpg
file load:0.023802757263183594
maeshori:0.03359986299983575
_______SEARCH START_______
 search time:0.1780810919990472
recog time: 0.0041162967681884766
 tensor([[ 6.6357, -6.9466, -2.5718]], device='cuda:0', grad_fn=<AddmmBackward>)
 Speed restriction 30
recog time:0.0224391209994792
about time:0.23584562399992137
0


**動作確認用**

In [0]:
def s_search(path,exam=True):
  from google.colab.patches import cv2_imshow
  import numpy as np
  import cv2
  # import selectivesearch
  import time 
  begin = time.time()
  print(f"\nimage path = {path}")
  for j in range(3):
    s = time.time()
    cut = rightCut(upCut(path),3,idx=j,verbose=False)
    img = simplize(cut,True)
    img = rb_filter(img,False)
    img = cv2.medianBlur(img,5)
    img = cv2.GaussianBlur(img,(17,17),0,0)
    print(f"maeshori:{time.time()-s}")
    print("_______SEARCH START_______")
    start = time.time()
    _, regions = selective_search(img.astype(np.float64), scale=3000, sigma=0, min_size=100)
    print(f" search time:{time.time() - start}")
    for r in regions:
      start = time.time()
      c = classify_torch(model,cut[r["y_"]:r["_y"],r["x_"]:r["_x"],:],exam)
      print(f"recog time:{time.time() - start}")
      if c!=3:
        cv2.rectangle(cut, (r["x_"], r["y_"]), (r["_x"], r["_y"]), (0, 0, 255), 2)
        cv2_imshow(cut)
    print(f"about time:{time.time() -begin}")

import os 
dire = "/content/drive/My Drive/3is_project/GTRSB_kaggle/micon_images/"
images = os.listdir(dire)
for i in images:
  img = dire + i
  s_search(img,False)

Output hidden; open in https://colab.research.google.com to view.